**Data Visualization course - winter semester 20/21 - FU Berlin**

*Tutorials adapted from the [Information Visualization](https://infovis.fh-potsdam.de/tutorials/) course at the FH Potsdam*


# Tutorial 6: Interaction techniques

Interactive capabilities make information visualizations come alive. They let viewers adjust a visualization to change its parameters, focus on an interesting aspect of a dataset, and get more detailed information about specific items. Arguably, the Jupyter environment itself is already an interactive environment. By adding interactive elements to visualizations, you can explore the data more directly and rapidly.

In this tutorial, you will get to know several techniques that provide interactivity to your notebooks and visualizations. 

✏️ *Remember to follow the pencils to get ideas for edits in this tutorial*



In [ ]:
# first we include the two libraries that we will work with
import pandas as pd
import altair as alt
from vega_datasets import data

In this tutorial, we are using our usual data.

In [ ]:
covid_data = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv", parse_dates=['date'])

code_lookup = pd.read_csv('country_lookup.csv')
countries = alt.topo_feature(data.world_110m.url, 'countries')

covid_data_sa = covid_data[ covid_data.continent == 'South America' ]
covid_data_sa = covid_data_sa.merge( code_lookup, left_on='iso_code', right_on='Alpha-3 code' )

## Zoom & pan

Visualizations created with Altair can be equipped with a simple zoom function that can be triggered either with the scroll gesture of a trackpad or the scrollwheel of the mouse. Depending on the chart type, this gives the viewer the option to gradually adjust which section of each axis is visible.

By adding the `.interactive()` method the axes of the chart become dynamic. Try zooming into the lineplot and notice how the labels on the axes move and adjust. You can also drag the visualization to change the current viewport. Double clicking anywhere resets the axes of the chart.

In [ ]:
alt.Chart(covid_data_sa).mark_line().encode(
    x='date:T',
    y='new_cases_smoothed',
    color='location'
).properties(
    width=800,
    height=400
).interactive()


✏️ *Now many lines overlap each other. Would it help to adjust their appearance, e.g., their size or opacity? You can set these default parameters as arguments to the `mark_line()` call. Have a look at the [mark property channels](https://altair-viz.github.io/user_guide/encoding.html#encoding-channels) provided by Altair*


## Details-on-demand

Because our ability to discern multiple channels in a visualization is limited, the visual variables that can be utilized to represent data dimensions are finite. We need to decide which information is encoded visually and which information can be made available interactively on demand. 

Tooltips constitute a classic technique to provide such additional, more detailed information on data elements. Your viewers will be able to reveal the details by hovering the mouse pointer over the respective visual elements.

To include a tooltip feature in above visualization, you need to tell Altair, which attributes should be included. You do this by adding a `tooltip` to the `encode()` method call:

In [ ]:
alt.Chart(covid_data_sa).mark_line().encode(
    x='date:T',
    y='new_cases_smoothed',
    color='location',
    tooltip=['location', 'new_cases_smoothed']
).properties(
    width=800,
    height=400
).interactive()


✏️ *Add other attributes to the tooltip that might be of interest for a reader*

## Interactive legend

At this point the scatterplot is not entirely instructive if we cannot distinguish between countries at the bottom of the chart. Maybe it would help to focus on a subset of data elements. Let's turn the legend into an interactive filter.

For this to work we need to create a selection element first, which we then add to the chart declaration as above, with the addition of changing the opacity:

In [ ]:
selection = alt.selection(type='multi', fields=['location'], bind='legend')

alt.Chart(covid_data_sa).mark_line().encode(
    x='date:T',
    y='new_cases_smoothed',
    color='location',
    tooltip=['location', 'new_cases_smoothed'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).properties(
    width=800,
    height=400
).add_selection(
    selection
)

## Input elements

If we would present a bigger range of countries, we would see that the legend would not scale well. The interactive legend only works well when you have a manageable number of entries, which is also true for using color for more than a handful of categories. There are just too many countries in the dataset to be handled via an interactive legend, if we would choose to visualize all of them. Therefore we could add an element which allows an easier selection of countries from our dataset:

In [ ]:
dropdown = alt.binding_select(options=covid_data_sa.location.unique(), name='Select a country: ')

selection = alt.selection(type='single', fields=['location'], bind=dropdown)

The following block is identical to the one above. Except now the chart will be accompanied by a country selector, and not an interactive legend.

In [ ]:
alt.Chart(covid_data_sa).mark_line().encode(
    x='date:T',
    y='new_cases_smoothed',
    color='location',
    tooltip=['location', 'new_cases_smoothed'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).properties(
    width=800,
    height=400
).add_selection(
    selection
)

## Linked views

Oftentimes we are working with complex datasets, which require selections to be made in the visualizations themselves. This allows us to specify data ranges according to particular interests as they emerge in the interaction with the visualizations.

The following example places two charts next to each other. Selecting a time range in the lineplot of new cases in the whole of South America on the left, will trigger the graduated symbols plot for the geographic distribution on the right.


In [ ]:
selection = alt.selection(type='interval', encodings=['x'])

line = alt.Chart(covid_data_sa).mark_line().encode(
    x='date',
    y='sum(new_cases_smoothed)',
).add_selection(
    selection
)

map = alt.Chart(countries).mark_geoshape(
    stroke='white',
    fill='#A9A9A9'
).project(
    type='mercator',
    scale=200,
    center=[-110, 15]
)

symbols = alt.Chart(covid_data_sa).mark_circle().encode(
    longitude='Longitude (average)',
    latitude='Latitude (average)',
    size=alt.Size('sum(new_cases_smoothed)', scale=alt.Scale(domain=[0, 0.05*covid_data_sa.new_cases_smoothed.sum()]))
).project(
    type='mercator',
    scale=200,
    center=[-110, 15]
).transform_filter(
    selection
)

line | (map + symbols)

## Sources

From the Altair documentation:
- [Encodings](https://altair-viz.github.io/user_guide/encoding.html)
- [Interactive Legend](https://altair-viz.github.io/gallery/interactive_legend.html)
- [Bindings, Selections, Conditions: Making Charts Interactive](https://altair-viz.github.io/user_guide/interactions.html)
- [Interactive Scatter Plot and Linked Layered Histogram](https://altair-viz.github.io/gallery/scatter_with_layered_histogram.html)
- [Interactive Crossfilter](https://altair-viz.github.io/gallery/interactive_layered_crossfilter.html)

